<a href="https://colab.research.google.com/github/DishaKushwah/custom-quiz-generator/blob/main/true_false_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#true false
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    T5Tokenizer,
    T5ForConditionalGeneration,
    pipeline
)
import nltk
import random
import re
import json
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

class TrueFalseQuestionGenerator:
    """
    A comprehensive True/False question generator using state-of-the-art models
    """

    def __init__(self, model_name: str = "google/flan-t5-large"):
        """
        Initialize the True/False Question Generator
        """
        self.model_name = model_name
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Load tokenizer and model
        if "t5" in model_name.lower():
            self.tokenizer = T5Tokenizer.from_pretrained(model_name)
            self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForCausalLM.from_pretrained(model_name)

        self.model.to(self.device)
        self.model.eval()

        # Initialize question generation pipeline
        self.question_pipeline = pipeline(
            "text2text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if torch.cuda.is_available() else -1
        )

    def extract_key_facts(self, context: str) -> List[str]:
        """
        Extract key facts from the context for question generation by splitting on punctuation
        """
        # Split the context into sentences using regex based on common punctuation
        sentences = re.split(r'(?<=[.!?])\s+', context)

        # Filter sentences that are likely to contain factual information
        factual_sentences = []
        for sentence in sentences:
            # Look for sentences with specific patterns that indicate facts
            if (len(sentence.split()) > 5 and
                any(keyword in sentence.lower() for keyword in
                    ['is', 'are', 'was', 'were', 'has', 'have', 'can', 'will',
                     'does', 'did', 'contains', 'includes', 'located', 'founded',
                     'established', 'invented', 'discovered', 'developed', 'used', 'use'])):
                factual_sentences.append(sentence.strip())

        return factual_sentences[:10]  # Limit to 10 key facts

    def generate_true_questions(self, context: str, num_questions: int = 5) -> List[Dict]:
        """
        Generate TRUE questions based on the context
        """
        facts = self.extract_key_facts(context)
        true_questions = []

        # Ensure we don't try to generate more questions than available facts
        num_to_generate = min(num_questions, len(facts))

        for i, fact in enumerate(facts[:num_to_generate]):
            # Create prompts for generating true/false questions
            prompts = [
                f"Convert this fact into a true/false question: {fact}",
                f"Create a statement that can be answered true or false based on: {fact}",
                f"Generate a true/false question from: {fact}"
            ]

            prompt = random.choice(prompts)

            try:
                # Generate question using the model
                response = self.question_pipeline(
                    prompt,
                    max_new_tokens=100, # Use max_new_tokens instead of max_length
                    num_return_sequences=1,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )

                generated_text = response[0]['generated_text'].strip()

                # Clean up the generated question
                question = self.clean_question(generated_text)

                if question:
                    true_questions.append({
                        'question': question,
                        'answer': True,
                        'explanation': f"Based on the context: {fact}",
                        'source_fact': fact
                    })

            except Exception as e:
                print(f"Error generating question {i+1}: {str(e)}")
                continue

        return true_questions

    def generate_false_questions(self, context: str, num_questions: int = 5) -> List[Dict]:
        """
        Generate FALSE questions by modifying facts from the context
        """
        facts = self.extract_key_facts(context)
        false_questions = []

        # To ensure we don't try to generate more questions than available facts
        num_to_generate = min(num_questions, len(facts))

        for i, fact in enumerate(facts[:num_to_generate]):
            # Create prompts for generating false statements that are plausible but contradict the fact
            prompts = [
                f"Create a plausible false statement based on this fact: {fact}",
                f"Generate a statement that sounds true but is false according to: {fact}",
                f"Make this statement false by changing a key detail: {fact}"
            ]

            prompt = random.choice(prompts)

            try:
                response = self.question_pipeline(
                    prompt,
                    max_new_tokens=100,
                    num_return_sequences=1,
                    temperature=0.8,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )

                generated_text = response[0]['generated_text'].strip()
                question = self.clean_question(generated_text)

                if question:
                    false_questions.append({
                        'question': question,
                        'answer': False,
                        'explanation': f"According to the context, the correct statement is: {fact}",
                        'source_fact': fact
                    })

            except Exception as e:
                print(f"Error generating false question {i+1}: {str(e)}")
                continue

        return false_questions

    def clean_question(self, text: str) -> str:
        """
        Clean and format the generated question
        """
        # Remove common prefixes
        text = re.sub(r'^(Question:|Q:|True/False:|Statement:)\s*', '', text, flags=re.IGNORECASE)

        # Ensure the question ends with proper punctuation
        text = text.strip()
        if not text.endswith(('.', '!', '?')):
            text += '.'

        # Capitalize first letter
        if text:
            text = text[0].upper() + text[1:]

        # Remove standalone "True" or "False" if it appears at the beginning
        text = re.sub(r'^(True|False)\.\s*', '', text, flags=re.IGNORECASE)
        text = re.sub(r'^(True|False)\?\s*', '', text, flags=re.IGNORECASE)


        return text if len(text) > 10 else None

    def generate_questions(self, context: str, num_questions: int = 10) -> List[Dict]:
        """
        Generate a mix of true and false questions
        """
        if not context.strip():
            raise ValueError("Context cannot be empty")

        print(f"Generating {num_questions} questions from context...")

        # Split questions evenly between true and false
        num_true = num_questions // 2
        num_false = num_questions - num_true

        true_questions = self.generate_true_questions(context, num_true)
        false_questions = self.generate_false_questions(context, num_false)

        # Combine and shuffle questions
        all_questions = true_questions + false_questions
        random.shuffle(all_questions)

        # Add question numbers
        for i, question in enumerate(all_questions, 1):
            question['id'] = i
        return all_questions

    def display_questions(self, questions: List[Dict], show_answers: bool = False):
        """
        Display questions in a formatted way
        """
        print("TRUE/FALSE QUESTIONS")

        for q in questions:
            print(f"\n{q['id']}. {q['question']}")

            if show_answers:
                answer_text = "TRUE" if q['answer'] else "FALSE"
                print(f"   Answer: {answer_text}")
                print(f"   Explanation: {q['explanation']}")

    def save_questions(self, questions: List[Dict], filename: str = "questions.json"):
        """
        Save questions to a JSON file
        """
        with open(filename.strip(), 'w', encoding='utf-8') as f:
            json.dump(questions, f, indent=2, ensure_ascii=False)
        print(f"Questions saved to {filename.strip()}")

# Example usage and testing
def main():
    """Main function to demonstrate the True/False Question Generator"""

    # Initialize the generator
    generator = TrueFalseQuestionGenerator()
     # Sample context for testing
    sample_context =input("Enter the context for true/false generation: ")
    try:
        num_questions = int(input("Number of questions to generate (default 5): ") or "5")
    except ValueError:
        num_questions = 5
    # Generate questions
    questions = generator.generate_questions(sample_context, num_questions)

    print("\n--- Generated Questions ---")
    # Display questions
    generator.display_questions(questions, show_answers=True)

    # Save questions
    generator.save_questions(questions, "questions.json")

if __name__ == "__main__":
    main()

Device set to use cpu


Enter the context for true/false generation: Currency is a system of money used for buying and selling goods and services. It has evolved from the barter system to coins, paper money, and now digital payments. International trade allows countries to exchange resources, goods, and services, often using foreign currencies like the US Dollar or Euro. Exchange rates determine how much one currency is worth compared to another. Organizations like the World Trade Organization (WTO) regulate global trade practices.
Number of questions to generate (default 5): 5
Generating 5 questions from context...

--- Generated Questions ---
TRUE/FALSE QUESTIONS

1. Exchange rates do not determine how much one currency is worth compared to another.
   Answer: FALSE
   Explanation: According to the context, the correct statement is: Exchange rates determine how much one currency is worth compared to another.

2. It has not evolved from the barter system to coins, paper money, and now digital payments.
   An